# DistilBERT Classifier using Hugging Face `transformers`

![](figures/finetuning-ii.png)

In [1]:
# pip install transformers

In [2]:
# pip install datasets

In [3]:
%load_ext watermark
%watermark --conda -p torch,transformers,datasets

torch       : 1.12.1
transformers: 4.23.1
datasets    : 2.6.1

conda environment: dl-fundamentals



In [4]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 1 Loading the Dataset

The IMDB movie review dataset consists of 50k movie reviews with sentiment label (0: negative, 1: positive).

## 1a) Load from `datasets` Hub

In [5]:
from datasets import list_datasets, load_dataset

In [6]:
# list_datasets()

In [7]:
imdb_data = load_dataset("imdb")
print(imdb_data)

Found cached dataset imdb (/home/raschka/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [8]:
imdb_data["train"][99]

{'text': "This film is terrible. You don't really need to read this review further. If you are planning on watching it, suffice to say - don't (unless you are studying how not to make a good movie).<br /><br />The acting is horrendous... serious amateur hour. Throughout the movie I thought that it was interesting that they found someone who speaks and looks like Michael Madsen, only to find out that it is actually him! A new low even for him!!<br /><br />The plot is terrible. People who claim that it is original or good have probably never seen a decent movie before. Even by the standard of Hollywood action flicks, this is a terrible movie.<br /><br />Don't watch it!!! Go for a jog instead - at least you won't feel like killing yourself.",
 'label': 0}

## 1b) Load from local directory

The IMDB movie review set can be downloaded from http://ai.stanford.edu/~amaas/data/sentiment/. After downloading the dataset, decompress the files.

A) If you are working with Linux or MacOS X, open a new terminal windowm cd into the download directory and execute

    tar -zxf aclImdb_v1.tar.gz

B) If you are working with Windows, download an archiver such as 7Zip to extract the files from the download archive.

C) Use the following code to download and unzip the dataset via Python

**Download the movie reviews**

In [9]:
import os
import sys
import tarfile
import time
import urllib.request

source = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
target = "aclImdb_v1.tar.gz"

if os.path.exists(target):
    os.remove(target)


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.0**2 * duration)
    percent = count * block_size * 100.0 / total_size

    sys.stdout.write(
        f"\r{int(percent)}% | {progress_size / (1024.**2):.2f} MB "
        f"| {speed:.2f} MB/s | {duration:.2f} sec elapsed"
    )
    sys.stdout.flush()


if not os.path.isdir("aclImdb") and not os.path.isfile("aclImdb_v1.tar.gz"):
    urllib.request.urlretrieve(source, target, reporthook)

In [10]:
if not os.path.isdir("aclImdb"):

    with tarfile.open(target, "r:gz") as tar:
        tar.extractall()

**Convert them to a pandas DataFrame and save them as CSV**

In [11]:
import os
import sys

import numpy as np
import pandas as pd
from packaging import version
from tqdm import tqdm

# change the `basepath` to the directory of the
# unzipped movie dataset

basepath = "aclImdb"

labels = {"pos": 1, "neg": 0}

df = pd.DataFrame()

with tqdm(total=50000) as pbar:
    for s in ("test", "train"):
        for l in ("pos", "neg"):
            path = os.path.join(basepath, s, l)
            for file in sorted(os.listdir(path)):
                with open(os.path.join(path, file), "r", encoding="utf-8") as infile:
                    txt = infile.read()

                if version.parse(pd.__version__) >= version.parse("1.3.2"):
                    x = pd.DataFrame(
                        [[txt, labels[l]]], columns=["review", "sentiment"]
                    )
                    df = pd.concat([df, x], ignore_index=False)

                else:
                    df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()
df.columns = ["text", "label"]

100%|███████████████████████████████████████████| 50000/50000 [00:55<00:00, 906.15it/s]


In [12]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

**Basic datasets analysis and sanity checks**

In [13]:
print("Class distribution:")
np.bincount(df["label"].values)

Class distribution:


array([25000, 25000])

In [14]:
text_len = df["text"].apply(lambda x: len(x.split()))
text_len.min(), text_len.median(), text_len.max() 

(4, 173.0, 2470)

**Split data into training, validation, and test sets**

In [15]:
df_train = df.iloc[:35_000]
df_val = df.iloc[35_000:40_000]
df_test = df.iloc[40_000:]

df_train.to_csv("train.csv", index=False, encoding="utf-8")
df_val.to_csv("validation.csv", index=False, encoding="utf-8")
df_test.to_csv("test.csv", index=False, encoding="utf-8")

**Load the dataset via `load_dataset`**

In [16]:
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "validation.csv",
        "test": "test.csv",
    },
)

print(imdb_dataset)

Using custom data configuration default-7d85f3fd1f344101


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/pandas/io/common.py:122: ResourceWarning: unclosed file <_io.BufferedReader name='/home/raschka/scratch/lightning-transformer-stuff/train.csv'>
  self.handle.detach()


0 tables [00:00, ? tables/s]

...

/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/raschka/.cache/huggingface/datasets/csv/default-7d85f3fd1f344101/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/pandas/io/common.py:122: ResourceWarning: unclosed file <_io.BufferedReader name='/home/raschka/scratch/lightning-transformer-stuff/test.csv'>
  self.handle.detach()


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})


# 2 Tokenization and Numericalization

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [18]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [19]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
del imdb_dataset

# 3 Finetuning DistilBERT

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)
model.to(device);

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [22]:
from transformers import Trainer, TrainingArguments
from datasets import load_metric

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": acc}

batch_size = 10
trainer_args = TrainingArguments(output_dir="distilbert-v1",
                                 num_train_epochs=3,
                                 evaluation_strategy="epoch",
                                 learning_rate=1e-5)

trainer = Trainer(model=model,
                  args=trainer_args,
                  compute_metrics=compute_metrics,
                  train_dataset=imdb_tokenized["train"],
                  eval_dataset=imdb_tokenized["validation"],
                  tokenizer=tokenizer)

trainer.train();

/tmp/ipykernel_23284/4127383031.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')
/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a f

Epoch,Training Loss,Validation Loss,Accuracy
1,0.204800,0.213792,{'accuracy': 0.9166}
2,0.154700,0.245530,{'accuracy': 0.92}
3,0.119300,0.233658,{'accuracy': 0.9222}


Saving model checkpoint to distilbert-v1/checkpoint-500
Configuration saved in distilbert-v1/checkpoint-500/config.json
Model weights saved in distilbert-v1/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-v1/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-v1/checkpoint-500/special_tokens_map.json
/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to distilbert-v1/checkpoint-1000
Configuration saved in distilbert-v1/checkpoint-1000/config.json
Model weights saved in distilbert-v1/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbert-v1/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbert-v1/checkpoint-1000/spe

In [23]:
outputs = trainer.predict(imdb_tokenized["train"])
outputs.metrics

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 35000
  Batch size = 32
/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'test_loss': 0.09039314836263657,
 'test_accuracy': {'accuracy': 0.972},
 'test_runtime': 274.6699,
 'test_samples_per_second': 127.426,
 'test_steps_per_second': 3.983}

In [25]:
outputs = trainer.predict(imdb_tokenized["validation"])
outputs.metrics

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5000
  Batch size = 32
/home/raschka/miniforge3/envs/dl-fundamentals/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'test_loss': 0.23365844786167145,
 'test_accuracy': {'accuracy': 0.9222},
 'test_runtime': 25.8022,
 'test_samples_per_second': 193.782,
 'test_steps_per_second': 6.085}

In [ ]:
outputs = trainer.predict(imdb_tokenized["test"])
outputs.metrics